In [1]:
import time

In [2]:
start_time = time.time()

In [13]:
from datetime import datetime
f'{datetime.fromtimestamp(time.time() - start_time):%M:%S}'

'08:09'

## German word vectors ~1.5Gb

In [5]:
!curl -O https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1218M  100 1218M    0     0  12.2M      0  0:01:39  0:01:39 --:--:-- 13.7MM


## English word vectors ~680Mb

In [30]:
!curl -O https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  650M  100  650M    0     0  8269k      0  0:01:20  0:01:20 --:--:-- 8567k0  0:01:25  0:00:46  0:00:39 9500k8163k      0  0:01:21  0:01:10  0:00:11 9749k:20  0:01:15  0:00:05 9429k


## Training Data

In [1]:
!curl -O https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.2M  100 23.2M    0     0  21.5M      0  0:00:01  0:00:01 --:--:-- 21.5M


In [2]:
!head -n 1 train.csv

Sport;21-Jähriger fällt wohl bis Saisonende aus. Wien – Rapid muss wohl bis Saisonende auf Offensivspieler Thomas Murg verzichten. Der im Winter aus Ried gekommene 21-Jährige erlitt beim 0:4-Heimdebakel gegen Admira Wacker Mödling am Samstag einen Teilriss des Innenbandes im linken Knie, wie eine Magnetresonanz-Untersuchung am Donnerstag ergab. Murg erhielt eine Schiene, muss aber nicht operiert werden. Dennoch steht ihm eine mehrwöchige Pause bevor.


## Test Data

In [3]:
!curl -O https://raw.githubusercontent.com/tblock/10kGNAD/master/test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2690k  100 2690k    0     0  5823k      0 --:--:-- --:--:-- --:--:-- 5823k


In [4]:
!head -n 1 test.csv

Wirtschaft;'Die Gewerkschaft GPA-djp lanciert den "All-in-Rechner" und findet, dass die Vertragsform auf die Führungsebene beschränkt gehört. Wien – Die Gewerkschaft GPA-djp sieht Handlungsbedarf bei sogenannten All-in-Verträgen. Für mehr Transparenz haben die Gewerkschaft nun unter www.allinrechner.at ein Online-Werkzeug lanciert, das Arbeitnehmern mit solchen Verträgen helfen soll, sich einen Überblick zu verschaffen. Es geht darum, ob die Differenz zum Grundgehalt nicht womöglich eine zu geringe ist. Schließlich sei ein All-in-Gehalt oft nur auf den ersten Blick von Vorteil, gab GPA-djp-Chef Wolfgang Katzian (SPÖ) am Montag bei einer Pressekonferenz in Wien zu bedenken. Allzu oft sei bei der All-in-Entlohnung nur schwer nachvollziehbar, wie viele Überstunden und Mehrleistungen damit tatsächlich abgegolten werden. Ein erster guter Schritt aus gewerkschaftlicher Sicht ist Katzian zufolge zwar eine seit Ende Vorjahr gültige Gesetzesänderung, die besagt, dass das Grundgehalt bei All-in-

## Spacy German model

### mainly for tokenisation

In [21]:
!python -m spacy download de_core_news_sm

usage: spacy download [-h] [-d] model [pip_args [pip_args ...]]

    Download compatible model from default download path using pip. Model
    can be shortcut, model name or, if --direct flag is set, full model name
    with version. For direct downloads, the compatibility check will be skipped.
    

positional arguments:
  model         Model to download (shortcut or name)
  pip_args      Additional arguments to be passed to `pip install` on model
                install

optional arguments:
  -h, --help    show this help message and exit
  -d, --direct  Force direct download of name + version


# download datasets from S3 (or Kaggle)

In [1]:
!ls -l data

total 840184
-rw-r--r--@  1 mattilyra  staff   42290090 Oct 24 10:29 10kgnad.zip
drwxr-xr-x  22 mattilyra  staff        704 Mar 18  2003 20news-bydate-test
drwxr-xr-x  22 mattilyra  staff        704 Mar 18  2003 20news-bydate-train
-rw-r--r--   1 mattilyra  staff   15343770 Sep  2 21:31 20news-bydate_py3.pkz
-rwxr-xr-x@  1 mattilyra  staff    1220776 Oct  2 08:53 Entity Recognition in Resumes.json
-rw-rw-rw-@  1 mattilyra  staff     504418 Oct 22 18:23 homer_life_hard.gif
-rw-r--r--@  1 mattilyra  staff     330445 Oct 24 10:29 resume-entities-for-ner.zip
drwx------@  4 mattilyra  staff        128 Oct 24 10:39 reviews-with-conditions
-rw-r--r--@  1 mattilyra  staff  354987126 Oct 24 10:29 reviews-with-conditions.zip


In [7]:
!head ./data/10kgnad/articles.csv

Etat;Die ARD-Tochter Degeto hat sich verpflichtet, ab August einer Quotenregelung zu folgen, die für die Gleichstellung von Regisseurinnen sorgen soll. In mindestens 20 Prozent der Filme, die die ARD-Tochter Degeto produziert oder mitfinanziert, sollen ab Mitte August Frauen Regie führen. Degeto-Chefin Christine Strobl folgt mit dieser Selbstverpflichtung der Forderung von Pro Quote Regie. Die Vereinigung von Regisseurinnen hatte im vergangenen Jahr eine Quotenregelung gefordert, um den weiblichen Filmschaffenden mehr Gehör und ökonomische Gleichstellung zu verschaffen. Pro Quote Regie kritisiert, dass, während rund 50 Prozent der Regie-Studierenden weiblich seien, der Anteil der Regisseurinnen bei Fernsehfilmen nur bei 13 bis 15 Prozent liege. In Österreich sieht die Situation ähnlich aus, auch hier wird von unterschiedlichen Seiten Handlungsbedarf angemahnt. Aber wie soll dieser aussehen? Ist die Einführung der Quotenregelung auch für die österreichische Film- und Fernsehlandschaft s

In [9]:
import pandas as pd

with open('./data/10kgnad/articles.csv', 'r') as fh:
    data = (line.split(';', 1) for line in fh)
    gnad = pd.DataFrame.from_records(data, columns=['category', 'text'])

In [13]:
gnad.category.value_counts()

Panorama         1678
Web              1677
International    1511
Wirtschaft       1411
Sport            1201
Inland           1015
Etat              668
Wissenschaft      573
Kultur            539
Name: category, dtype: int64

In [15]:
gnad.head(n=25)

,category,text
0,Etat,"Die ARD-Tochter Degeto hat sich verpflichtet, ..."
1,Etat,App sei nicht so angenommen worden wie geplant...
2,Etat,"'Zum Welttag der Suizidprävention ist es Zeit,..."
3,Etat,Mitarbeiter überreichten Eigentümervertretern ...
4,Etat,Service: Jobwechsel in der Kommunikationsbranc...
5,Etat,Was Sie über diese Woche wissen sollten - und ...
6,Etat,Was Sie über diese Woche wissen sollten – und ...
7,Etat,Medienjournalist möchte eine eigene Plattform ...
8,Etat,Autorenkollektiv soll als feste Redaktion zusa...
9,Etat,Nachrichten-App nach drei Jahren an Finanzieru...


In [20]:
import spacy

spc = spacy.load('de_core_news_sm')

In [19]:
spc.tokenizer(gnad.iloc[0].text)

Die